In [2]:

# Divides a set on a specific column. Can handle numeric or nominal values
#pseudocode for forming individual trees
def divideset(rows,column,value):
   # Make a function that tells us if a row is in the first group (true) or the second group (false)
   split_function=None
   if isinstance(value,int) or isinstance(value,float): # check if the value is a number i.e int or float
      split_function=lambda row:row[column]>=value
   else:
      split_function=lambda row:row[column]==value
   
   # Divide the rows into two sets and return them
   set1=[row for row in rows if split_function(row)]
   set2=[row for row in rows if not split_function(row)]
   return (set1,set2)


In [10]:
import numpy as np
import pandas as pd # Data frames
import matplotlib.pyplot as plt # Visuals
from sklearn.model_selection import train_test_split # Create training and test sets
from sklearn.neighbors import KNeighborsClassifier # Kth Nearest Neighbor
from sklearn.tree import DecisionTreeClassifier # Decision Trees
from sklearn.tree import export_graphviz # Extract Decision Tree visual
from sklearn import tree #decision trees
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.neural_network import MLPClassifier # Neural Networks
from sklearn.metrics import roc_curve # ROC Curves
from sklearn.metrics import auc # AUC 
from urllib.request import urlopen # Get data from UCI Machine Learning Repository
plt.style.use('ggplot')

Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Hungarian_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
Switzerland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data'

np.set_printoptions(threshold=np.nan)#makes it so that we can see a whole array when we output it

names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heart disease(by angiography)']

ClevelandHeartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)#gets Cleveland data
HungarianHeartDisease = pd.read_csv(urlopen(Hungarian_data_URL), names = names)#gets Hungary data
SwitzerlandHeartDisease = pd.read_csv(urlopen(Switzerland_data_URL), names = names)#gets Switzerland data
datatemp = [ClevelandHeartDisease, HungarianHeartDisease, SwitzerlandHeartDisease]#combines all arrays into a list

#heartDisease.set_index(['id_number'], inplace = True)
#I don't know what that did, but it was from Raoul's code and it was giving me bugs so I got rid of it
data = pd.concat(datatemp)#combines list into one array
del data['ca']
del data['slope']
del data['thal']#these delete the columns that have a lot of ? values and so aren't very useful to us

data = data.replace('?',np.nan)#convert ? values to np.nan

for item in data:#converts everything to floats
    data[item] = pd.to_numeric(data[item])

def normalize(heartDisease, toNormalize):#normalizes 
    result = heartDisease.copy()
    for item in heartDisease.columns:
        if (item in toNormalize):
            max_value = heartDisease[item].max()
            min_value = heartDisease[item].min()
            result[item] = (heartDisease[item] - min_value) / (max_value - min_value)
    return result
toNormalize = ['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak']#columns to normalize
data = normalize(data, toNormalize)
data = data.dropna()#gets rid of nan values
counter = 0
for i in range(1,5):#replaces angiography numbers with 1
    data['heart disease(by angiography)'] = data['heart disease(by angiography)'].replace(i,1)


In [11]:
#count = 0
#for item in data:
 #   for i in data[item]:
  #      count += (i=='?')
   # print(count)
    #count = 0


In [12]:
train0, test0 = train_test_split(data, test_size = 0.20, random_state = 42)# splits data into test set and train set

data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,heart disease(by angiography)
0,0.000000,1,0.333333,0.416667,0.218905,0.0,2.0,0.961538,0.0,0.342105,0
1,0.021739,1,0.333333,0.333333,0.402985,0.0,0.0,0.769231,0.0,0.342105,0
3,0.043478,0,0.000000,0.750000,0.393035,0.0,1.0,0.846154,0.0,0.342105,0
4,0.065217,0,0.333333,0.166667,0.363184,0.0,1.0,0.692308,0.0,0.342105,0
5,0.086957,0,0.333333,0.208333,0.328358,0.0,0.0,0.807692,0.0,0.342105,0
6,0.086957,1,0.333333,0.250000,0.373134,0.0,0.0,0.953846,0.0,0.342105,0
7,0.086957,1,0.333333,0.375000,0.421227,0.0,0.0,0.730769,0.0,0.342105,0
8,0.108696,1,0.666667,0.333333,0.494196,0.0,0.0,0.961538,0.0,0.342105,0
9,0.130435,0,0.333333,0.416667,0.266998,0.0,0.0,1.000000,0.0,0.342105,0
10,0.130435,1,0.333333,0.583333,0.354892,0.0,1.0,0.830769,0.0,0.342105,0


In [13]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train0[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']], train0['heart disease(by angiography)'])
predictions = clf.predict(test0[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']])
predictright = 0
predictions.shape[0]
for i in range(0,predictions.shape[0]-1):
    if (predictions[i]== test0.iloc[i][10]):
        predictright +=1
rightpercent = predictright/predictions.shape[0]
rightpercent

0.9473684210526315

In [21]:
#PUTTING TOGETHER A DATASET FOR THE DATABASES THAT DO NOT HAVE ONLY 1S FOR HEART DISEASE
datatemp2 = [ClevelandHeartDisease, SwitzerlandHeartDisease]
data2 = pd.concat(datatemp2)
del data2['ca']
del data2['slope']
del data2['thal']#these delete the columns that have a lot of ? values and so aren't very useful to us

data2 = data2.replace('?',np.nan)#convert ? values to np.nan

for item in data2:#converts everything to floats
    data2[item] = pd.to_numeric(data2[item])

data2 = normalize(data2, toNormalize)
data2 = data2.dropna()#gets rid of nan values    
train2, test2 = train_test_split(data2, test_size = 0.20, random_state = 42)#splits data into test set and training set

#CREATING AND EDITING DATASET FOR JUST HUNGARIAN HEART DISEASE
del HungarianHeartDisease['ca']
del HungarianHeartDisease['slope']
del HungarianHeartDisease['thal']#these delete the columns that have a lot of ? values and so aren't very useful to us
HungarianHeartDisease = HungarianHeartDisease.replace('?',np.nan)#convert ? values to np.nan
for item in HungarianHeartDisease:#converts everything to floats
    HungarianHeartDisease[item] = pd.to_numeric(HungarianHeartDisease[item])
HungarianHeartDisease = normalize(HungarianHeartDisease, toNormalize)
HungarianHeartDisease = HungarianHeartDisease.dropna()#gets rid of nan values
trainH, testH = train_test_split(HungarianHeartDisease, test_size = 0.20, random_state = 42)# splits data into test set and train set



KeyError: 'ca'

In [22]:
clfH = tree.DecisionTreeClassifier()
clfH = clfH.fit(trainH[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']], trainH['heart disease(by angiography)'])
predictionsH = clfH.predict(testH[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']])
predictrightH = 0
predictionsH.shape[0]
for i in range(0,predictionsH.shape[0]-1):
    if (predictionsH[i]== testH.iloc[i][10]):
        predictrightH +=1
rightpercentH = predictrightH/predictionsH.shape[0]
rightpercentH

NameError: name 'trainH' is not defined

In [16]:
clf2 = tree.DecisionTreeClassifier()
clf2 = clf2.fit(train2[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']], train2['heart disease(by angiography)'])
predictions2 = clf2.predict(test2[['age', 'sex', 'cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak']])
predictright2 = 0
for i in range(0,predictions2.shape[0]-1):
    if (predictions2[i]== test2.iloc[i][10]):
        predictright2 +=1
rightpercent2 = predictright2/predictions2.shape[0]
rightpercent2

0.5645161290322581

In [17]:
predictions2

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 3, 0, 1, 3, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 3, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 3, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1], dtype=int64)

In [18]:
data2


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,heart disease(by angiography)
0,0.000000,1,0.333333,0.416667,0.218905,0.0,2.0,0.961538,0.0,0.342105,0
1,0.021739,1,0.333333,0.333333,0.402985,0.0,0.0,0.769231,0.0,0.342105,0
3,0.043478,0,0.000000,0.750000,0.393035,0.0,1.0,0.846154,0.0,0.342105,0
4,0.065217,0,0.333333,0.166667,0.363184,0.0,1.0,0.692308,0.0,0.342105,0
5,0.086957,0,0.333333,0.208333,0.328358,0.0,0.0,0.807692,0.0,0.342105,0
6,0.086957,1,0.333333,0.250000,0.373134,0.0,0.0,0.953846,0.0,0.342105,0
7,0.086957,1,0.333333,0.375000,0.421227,0.0,0.0,0.730769,0.0,0.342105,0
8,0.108696,1,0.666667,0.333333,0.494196,0.0,0.0,0.961538,0.0,0.342105,0
9,0.130435,0,0.333333,0.416667,0.266998,0.0,0.0,1.000000,0.0,0.342105,0
10,0.130435,1,0.333333,0.583333,0.354892,0.0,1.0,0.830769,0.0,0.342105,0
